# Clustering Crypto

In [34]:
# Initial imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [3]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
r = requests.get(url)
crypto_data = r.json()['Data']

In [4]:
# Create a DataFrame 
crypto_df = pd.DataFrame(crypto_data).T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,0.000244,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing

In [5]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]

In [6]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df.IsTrading.eq(True)]
crypto_df.tail(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
ELY,Elysian,N/A,True,N/A,260000000,-1
EM,Eminer,N/A,True,N/A,2100000000,-1
ENCRYPG,EncrypGen,N/A,True,N/A,70938084.472832,-1
TSL,Energo,N/A,True,N/A,1000000000,-1
ENG,Enigma,N/A,True,N/A,150000000,-1
ENJ,Enjin Coin,N/A,True,N/A,1000000000,-1
ESS,Essentia,N/A,True,N/A,1755313373,-1
ETH,Ethereum,Ethash,True,PoW,120217714.749,-1
FUEL,Etherparty,N/A,True,N/A,1000000000,-1
ARCONA,Arcona,N/A,True,N/A,15181707.013085,-1


In [7]:
# Keep only cryptocurrencies with a working algorithm
crypto_df= crypto_df[crypto_df.Algorithm != 'N/A']
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,42
365,365Coin,X11,True,PoW/PoS,0,-1
404,404Coin,Scrypt,True,PoW/PoS,0,-1
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0
...,...,...,...,...,...,...
DCR,Decred,BLAKE256,True,PoW/PoS,13900419.888807,21000000
DBC,DeepBrain Chain,NEP-5,True,N/A,10000000000,-1
DERO,Dero,CryptoNight,True,PoW,18400000,-1
DOGE,Dogecoin,Scrypt,True,PoW,133851876383.705231,-1


In [8]:
# Remove the "IsTrading" column
crypto_df.drop(['IsTrading'], axis = 1, inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,42
365,365Coin,X11,PoW/PoS,0,-1
404,404Coin,Scrypt,PoW/PoS,0,-1
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
...,...,...,...,...,...
DCR,Decred,BLAKE256,PoW/PoS,13900419.888807,21000000
DBC,DeepBrain Chain,NEP-5,N/A,10000000000,-1
DERO,Dero,CryptoNight,PoW,18400000,-1
DOGE,Dogecoin,Scrypt,PoW,133851876383.705231,-1


In [9]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)

In [10]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df= crypto_df[crypto_df.TotalCoinsMined != 0]
crypto_df

for i in crypto_df['MaxSupply']
    print i

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,42
NSR,NuShares,PoS,PoS,6175418477.9692,0
TRI,Triangles Coin,X13,PoW/PoS,191623.903871,0
CMTC,CometCoin,Scrypt,PoW,872830,0
CHAT,OpenChat,Scrypt,PoW/PoS,1000000000,-1
...,...,...,...,...,...
DCR,Decred,BLAKE256,PoW/PoS,13900419.888807,21000000
DBC,DeepBrain Chain,NEP-5,N/A,10000000000,-1
DERO,Dero,CryptoNight,PoW,18400000,-1
DOGE,Dogecoin,Scrypt,PoW,133851876383.705231,-1


In [11]:
# Drop rows where there are 'N/A' text values
crypto_df= crypto_df[crypto_df.ProofType != 'N/A']
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,42
NSR,NuShares,PoS,PoS,6175418477.9692,0
TRI,Triangles Coin,X13,PoW/PoS,191623.903871,0
CMTC,CometCoin,Scrypt,PoW,872830,0
CHAT,OpenChat,Scrypt,PoW/PoS,1000000000,-1
...,...,...,...,...,...
DASH,Dash,X11,PoW/PoSe,10662761.421781,18900000
DCR,Decred,BLAKE256,PoW/PoS,13900419.888807,21000000
DERO,Dero,CryptoNight,PoW,18400000,-1
DOGE,Dogecoin,Scrypt,PoW,133851876383.705231,-1


In [12]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinname_df = crypto_df.iloc[:,0]
coinname_df

42             42 Coin
NSR           NuShares
TRI     Triangles Coin
CMTC         CometCoin
CHAT          OpenChat
             ...      
DASH              Dash
DCR             Decred
DERO              Dero
DOGE          Dogecoin
ETH           Ethereum
Name: CoinName, Length: 136, dtype: object

In [13]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
clean_crypto_df = crypto_df.drop(columns='CoinName')
clean_crypto_df

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,Scrypt,PoW/PoS,41.999952,42
NSR,PoS,PoS,6175418477.9692,0
TRI,X13,PoW/PoS,191623.903871,0
CMTC,Scrypt,PoW,872830,0
CHAT,Scrypt,PoW/PoS,1000000000,-1
...,...,...,...,...
DASH,X11,PoW/PoSe,10662761.421781,18900000
DCR,BLAKE256,PoW/PoS,13900419.888807,21000000
DERO,CryptoNight,PoW,18400000,-1
DOGE,Scrypt,PoW,133851876383.705231,-1


In [14]:
# Create dummy variables for text features
X = pd.get_dummies(clean_crypto_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,MaxSupply,Algorithm_Autolykos,Algorithm_BEP-2,Algorithm_BEP-20 Token,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2B + SHA3,Algorithm_Blake2b,Algorithm_C31,...,ProofType_PoW/PoSe,ProofType_PoW/nPoS,ProofType_ProgPoW/PoS,ProofType_Proof of Authority,ProofType_Proof-of-Work,ProofType_SPoS,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW,ProofType_dPoW/PoW
42,41.999952,42,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NSR,6175418477.9692,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TRI,191623.903871,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CMTC,872830,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHAT,1000000000,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Standardize data
data_scaler = StandardScaler()
data_scaler.fit(X)
X_scaled = data_scaler.transform(X)
X_scaled[:1]

array([[-0.08786899, -0.09220965, -0.0860663 , -0.0860663 , -0.0860663 ,
        -0.12216944, -0.0860663 , -0.0860663 , -0.12216944, -0.12216944,
        -0.15018785, -0.0860663 , -0.0860663 , -0.0860663 , -0.23294541,
        -0.12216944, -0.0860663 , -0.0860663 , -0.0860663 , -0.29664794,
        -0.0860663 , -0.0860663 , -0.23294541, -0.0860663 , -0.0860663 ,
        -0.12216944, -0.0860663 , -0.0860663 , -0.0860663 , -0.0860663 ,
        -0.0860663 , -0.0860663 , -0.15018785, -0.0860663 , -0.0860663 ,
        -0.12216944, -0.19536617, -0.0860663 , -0.0860663 , -0.15018785,
        -0.12216944, -0.29664794, -0.12216944, -0.0860663 , -0.0860663 ,
        -0.0860663 ,  2.1602469 , -0.0860663 , -0.0860663 , -0.0860663 ,
        -0.0860663 , -0.19536617, -0.0860663 , -0.19536617, -0.12216944,
        -0.0860663 , -0.0860663 , -0.0860663 , -0.0860663 , -0.0860663 ,
        -0.25      , -0.0860663 , -0.0860663 , -0.12216944, -0.12216944,
        -0.0860663 , -0.32510161, -0.0860663 , -0.0

### Reducing Dimensions Using PCA

In [16]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [17]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame( data=crypto_pca, columns=["principal component 1", 
                              "principal component 2", 
                              "principal component 3"], index=clean_crypto_df.index
)
df_crypto_pca

,principal component 1,principal component 2,principal component 3
42,0.179966,-1.304388,-1.411762
NSR,0.658958,-1.193749,-0.261227
TRI,0.589761,-1.966520,-1.699170
CMTC,-0.825948,0.493826,-0.380201
CHAT,0.179971,-1.304384,-1.411762
...,...,...,...
DASH,0.431887,-1.180894,-0.506618
DCR,0.556323,-1.979110,-1.809579
DERO,-0.890643,0.893956,0.149526
DOGE,-0.825283,0.494316,-0.380229


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [18]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
# Looking for the best k
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create the Elbow Curve using hvPlot
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [31]:
# Initialize the K-Means model
crypto_model = KMeans(n_clusters=5, random_state=5)
# Fit the model
crypto_model.fit(df_crypto_pca)
# Predict clusters
predictions = crypto_model.predict(df_crypto_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
final_crpyto_table = pd.concat([crypto_df, df_crypto_pca], axis = 1)
final_crpyto_table["Class"] = crypto_model.labels_
final_crpyto_table.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply,principal component 1,principal component 2,principal component 3,Class
42,42 Coin,Scrypt,PoW/PoS,41.999952,42,0.179966,-1.304388,-1.411762,0
NSR,NuShares,PoS,PoS,6175418477.9692,0,0.658958,-1.193749,-0.261227,0
TRI,Triangles Coin,X13,PoW/PoS,191623.903871,0,0.589761,-1.966520,-1.699170,0
CMTC,CometCoin,Scrypt,PoW,872830,0,-0.825948,0.493826,-0.380201,1
CHAT,OpenChat,Scrypt,PoW/PoS,1000000000,-1,0.179971,-1.304384,-1.411762,0


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [37]:
# Scale data to create the scatter plot
final_crpyto_table['TotalCoinsMined'] = np.ln(final_crpyto_table['TotalCoinsMined'])
final_crpyto_table['MaxSupply'] = np.ln(final_crpyto_table['MaxSupply'])

AttributeError: module 'numpy' has no attribute 'ln'

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
final_crpyto_table.hvplot.scatter(x='TotalCoinsMined', y='MaxSupply', by="Class")

#### Table of Tradable Cryptocurrencies

In [39]:
# Table with tradable cryptos
for i in crypto_df['MaxSupply']:
    

In [ ]:
# Print the total number of tradable cryptocurrencies
